<a href="https://colab.research.google.com/github/Communist02/BIN-22-1_Mazur_Denis_ML/blob/master/%D0%94%D0%97_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pandas scikit-learn nltk pymorphy3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 63.0 MB/s eta 0:00:00


In [3]:
import nltk
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy3 import MorphAnalyzer
import math

nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('stopwords')


class Processing:
    def tokenize(self, text: str) -> list[str]:
        return word_tokenize(text)

    def lematize(self, tokens: list[str]) -> list[str]:
        morph3 = MorphAnalyzer()
        lemmatized_words = [morph3.parse(
            word)[0].normal_form for word in tokens]
        return lemmatized_words

    def stemming(self, tokens: list[str]) -> list[str]:
        stemmer = SnowballStemmer("english")
        lemmatized_words = [stemmer.stem(word) for word in tokens]
        return lemmatized_words

    def vectorize(self, tokens: list[str]) -> list[int]:
        dict_vectors = {}
        result = []
        for word in tokens:
            if word in dict_vectors.keys():
                result.append(dict_vectors[word])
            else:
                dict_vectors[word] = len(dict_vectors)
                result.append(dict_vectors[word])
        return result

    def vectorize_dict(self, tokens: list[str]) -> list[int]:
        dict_vectors = {}
        for word in tokens:
            if word not in dict_vectors.keys():
                dict_vectors[word] = len(dict_vectors)
        return dict_vectors

    def delete_stop_words(self, tokens: list[str]) -> list[int]:
        stop_words = set(stopwords.words('english')).union(
            ['.', ',', ':', '?', '!', '(', ')'])
        return [word for word in tokens if word not in stop_words]

    def bag_of_words(self, tokens: list[str]) -> dict[str]:
        dict_words = {}
        for word in tokens:
            dict_words[word] = dict_words.setdefault(word, 0) + 1
        return dict_words

    def tf(self, tokens: list[str]) -> dict[str]:
        dict_words = self.bag_of_words(tokens)
        for word in dict_words:
            dict_words[word] /= len(tokens)
        return dict_words

    def idf(self, texts: list[list[str]]) -> dict[str]:
        dict_words = {}
        big_text = []
        for text in texts:
            big_text += list(set(text))
        for word in set(big_text):
            dict_words[word] = math.log(len(texts) / big_text.count(word))
        return dict_words

    def tf_idf(self, texts: list[list[str]], indexText: int) -> dict[str]:
        tf = self.tf(texts[indexText])
        idf = self.idf(texts)
        dict_words = {}
        for word in tf:
            dict_words[word] = tf[word] * idf[word]
        return dict_words

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# The AG News Corpus is a popular dataset commonly used for text classification tasks in Natural Language Processing (NLP). It consists of news articles collected from the AG's corpus of news articles on the web, categorized into four classes: World, Sports, Business, and Science/Technology. Each article is accompanied by a title and a short description, making it suitable for tasks like topic classification and sentiment analysis. With its diverse range of topics and well-labeled categories, the AG News Corpus serves as a valuable resource for training and evaluating machine learning models in various NLP applications.

# Description:

# Dataset: AG News Corpus
# Source: AG's corpus of news articles on the web.
# Content: News articles categorized into World, Sports, Business, and Science/Technology.
# Labels: Four class labels representing different news categories.
# Scope: Covers a broad range of current events and topics.
# Size: Typically contains thousands of articles.
# Language: Primarily in English.

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('train.csv')
pc = Processing()

y = df['Class Index']
x = df.copy().drop('Class Index', axis=1)

In [ ]:
df

In [ ]:
for index, row in df.iterrows():
    text = row['Title']
    tokens = pc.tokenize(text)  # Токенизация
    tokens = pc.lematize(tokens)  # Лемматизация
    tokens = pc.delete_stop_words(tokens)  # Удаление стоп слов

    row['Title'] = ' '.join(tokens)

    text = row['Description']
    tokens = pc.tokenize(text)  # Токенизация
    tokens = pc.lematize(tokens)  # Лемматизация
    tokens = pc.delete_stop_words(tokens)  # Удаление стоп слов

    row['Description'] = ' '.join(tokens)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

x_train, x_valid, y_train, y_valid = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42
)